## Object Pool

- __Type:__ Creational
- __Popularity: ★★☆☆☆__
- __Complexity: ★★★☆☆__

### Intent:
__Object Pool__ is a creational design pattern that manages a pool of reusable objects to improve performance and resource utilization. It allows clients to "borrow" objects from the pool, use them, and return them to the pool instead of creating and destroying objects repeatedly.

### Problem:
Creating and destroying objects, especially those that are expensive to initialize (like database connections, thread objects, or network connections), can be resource-intensive and harm performance. When these objects are needed frequently but for short periods, the overhead of initialization and destruction becomes significant.

Additionally, there may be scenarios where we need to limit the number of objects that can exist at any given time to manage resources effectively.

### Solution:
The Object Pool pattern maintains a set of initialized objects ready for use, rather than allocating and destroying them on demand. When a client needs an object, it requests one from the pool. If a free object is available, it is returned. When the client is done with the object, it returns it to the pool rather than destroying it.

The pattern typically involves a pool manager class that maintains two collections of objects: available and in-use. The manager handles the logic for acquiring, releasing, and potentially creating new objects when needed. The reusable objects themselves usually implement a reset method to clear their state before being reused.

### Diagram:

```mermaid
classDiagram
    class ObjectPool {
        -availableObjects
        -inUseObjects
        +acquireObject()
        +releaseObject(object)
    }
    class ReusableObject {
        +reset()
        +operation()
    }
    class Client {
        +useObject()
    }
    ObjectPool o-- ReusableObject : manages
    Client --> ObjectPool : requests/returns
    Client --> ReusableObject : uses
```

### Example code:

In [ ]:
# A car rental example of the Object Pool pattern


class Car:
    def __init__(self, id: int) -> None:
        self.id = id

    def reset(self) -> None:
        # Reset the object's state before returning to pool
        print(f"Resetting car {self.id}")

    def __str__(self) -> str:
        return f"Car(id={self.id})"


class Garage:
    def __init__(self, size: int) -> None:
        # Initialize the pool with a fixed number of objects
        self._available = [Car(i) for i in range(size)]
        self._in_use = []

    def acquire(self) -> Car:
        # Acquire an available object from the pool
        if not self._available:
            raise Exception("No cars available")
        car = self._available.pop()
        self._in_use.append(car)
        return car

    def release(self, car: Car) -> None:
        # Return an object to the pool
        car.reset()  # Reset the object before reuse
        self._in_use.remove(car)
        self._available.append(car)

    def __str__(self) -> str:
        return f"Garage(available={len(self._available)}, in_use={len(self._in_use)})"

In [ ]:
# Usage example
if __name__ == "__main__":
    # Create a pool with 2 cars
    garage = Garage(2)
    print(garage)

    # Acquire first car
    car1 = garage.acquire()
    print(f"Acquired: {car1}")
    print(garage)

    # Acquire second car
    car2 = garage.acquire()
    print(f"Acquired: {car2}")
    print(garage)

    # Return first car to the pool
    garage.release(car1)
    print(f"Released: {car1}")
    print(garage)

    # Acquire another car (will get the one we just released)
    car3 = garage.acquire()
    print(f"Acquired: {car3}")
    print(garage)

Garage(available=2, in_use=0)
Acquired: Car(id=1)
Garage(available=1, in_use=1)
Acquired: Car(id=0)
Garage(available=0, in_use=2)
Resetting car 1
Released: Car(id=1)
Garage(available=1, in_use=1)
Acquired: Car(id=1)
Garage(available=0, in_use=2)


### Real-world analogies:

1. Library system:
   A library maintains a collection of books that can be borrowed by readers. Instead of buying a new book every time you want to read something, you borrow a book from the library, read it, and return it so others can use it too. The library is the object pool, and the books are the reusable objects.

2. Car rental service:
   A car rental company maintains a fleet of cars that customers can rent. When a customer needs a car, they rent one from the available pool. When they're done, they return the car to the rental company, which resets it (cleans, refuels) before making it available to the next customer. The rental company is the object pool, and the cars are the reusable objects.

### When to use:

- When objects are expensive to create (in terms of time or resources)
- When you need many instances of an object for short durations
- When you need to limit the number of instances that can exist simultaneously
- For managing shared resources like database connections, thread pools, or network connections
- When the rate of object creation and destruction is high

### Python-specific implementation notes:

- Python's garbage collection handles memory management automatically, but Object Pool is still valuable for expensive-to-create resources
- The `contextlib.contextmanager` decorator can be used to create a context manager for automatically returning objects to the pool
- Python's `queue.Queue` class can be used as a thread-safe implementation of the object pool
- Libraries like `dbutils.pooled_db` for database connection pooling and `multiprocessing.Pool` for process pooling implement this pattern
- The `__del__` method can be used to automatically return objects to the pool when they go out of scope

### Related patterns:

- Singleton: Often the Object Pool itself is implemented as a Singleton
- Factory Method: Can be used to create objects for the pool
- Flyweight: Shares similar goals of reusing objects, but Flyweight focuses on sharing to reduce memory usage while Object Pool focuses on reuse to improve performance
- Prototype: Can be used with Object Pool to create new objects for the pool by cloning a prototype